In [11]:
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, TimestampType, LongType, ArrayType
import json
from datetime import datetime

class search_in_sparksql(object):
    def __init__(self,):
        # warehouse_location = "hdfs://data/"
        
        # Set the warehouse location
        warehouse_location = "file:///home/hx152/Project/tweet_project/Dataset"
        metastore_db_location = "/home/hx152/Project/tweet_project/Dataset/metastore_db"

        self.spark = SparkSession.builder \
            .enableHiveSupport() \
            .config("spark.sql.warehouse.dir", warehouse_location) \
            .config("spark.sql.legacy.createHiveTableByDefault", False) \
            .config("javax.jdo.option.ConnectionURL", f"jdbc:derby:;databaseName={metastore_db_location};create=true") \
            .appName("UserInformation").getOrCreate()
        sc = self.spark.sparkContext
        self.sparksql = SQLContext(sc)
        
    def search_by_name(self, search_string, table_name="user_data.user_basic"):
        # Escape single quotes in the search string to avoid SQL injection
        search_string = search_string.replace("'", "''")

        # Execute the SQL query
        query = f"""
            SELECT *
            FROM {table_name}
            WHERE name LIKE '%{search_string}%' OR screen_name LIKE '%{search_string}%'
        """
        result = self.spark.sql(query)
    
        # Display the result
        return result

In [12]:
name = 'John'
ss = search_in_sparksql()
ss.search_by_name(name)

23/04/25 22:54:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


DataFrame[tweet_create_time: timestamp, tweet_id: bigint, user_id: bigint, name: string, screen_name: string, location: string, url: string, description: string, verified: boolean, created_at: timestamp, followers_count: int, friends_count: int, listed_count: int, favourites_count: int, statuses_count: int]